In [1]:
from utils.FeatureCalculator import FeatureCalculator
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))


compo_elem = ["Li", "Fe", "Cu", "P"]  # Your data


# # = [atomic_numbers_dict[elem] for elem in compo_elem]
# print(comp_num)

ele_frac = [0.3, 0.4, 0.3, 0.0000001]  # Your data

calculator = FeatureCalculator(compo_elem, ele_frac)
features = calculator.calculate_features()

print(features)

ModuleNotFoundError: No module named 'utils'

In [ ]:
import os
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from utils.FeatureCalculator import FeatureCalculator

T_fname = "MultiTaskModel_NiCrMoTiFe_KW131_at_pct.xlsx"
Features_Sel = ["delta_a", "Hmix", "VEC", "sigma_bulk_modulus"]

# Load data
T = pd.read_excel(T_fname)
display(T.head())

,Index,Ni,Cr,Mo,Ti,Fe,FORMULA,delta_a,Hmix,VEC,sigma_bulk_modulus
0,0,43.8,38.3,2.44,1.04,14.5,Ni43.8 Cr38.3 Mo2.44 Ti1.04 Fe14.5,0.022532,-6.600175,8.019584,14.327778
1,1,39.5,39.9,3.47,1.54,15.5,Ni39.5 Cr39.9 Mo3.47 Ti1.54 Fe15.5,0.026940,-6.783670,7.860875,16.085454
2,2,36.7,40.2,4.64,2.30,16.2,Ni36.7 Cr40.2 Mo4.64 Ti2.3 Fe16.2,0.031816,-7.109403,7.745302,18.041600
3,3,33.9,39.9,5.95,3.21,17.1,Ni33.9 Cr39.9 Mo5.95 Ti3.21 Fe17.1,0.036503,-7.447112,7.632820,20.055657
4,4,31.7,38.4,7.92,4.64,17.4,Ni31.7 Cr38.4 Mo7.92 Ti4.64 Fe17.4,0.042348,-8.086359,7.522287,22.809297


In [ ]:
formula = T["FORMULA"]
alloy_num = len(formula)

# Extract component_num, components, and fractions
components = []
fractions = []
component_num = []

for i in range(alloy_num):
    formula_temp = formula[i]
    formula_temp_split = re.split("\s+", formula_temp)
    component_num.append(len(formula_temp_split))
    components.append(" ".join([re.sub(r"\d+", "", s)
                      for s in formula_temp_split]))
    fractions.append(" ".join([re.sub(r"\D", "", s)
                     for s in formula_temp_split]))

# Calculate features
features = np.zeros((alloy_num, 13))
components_split = []
fractions_split = []

for i in range(alloy_num):
    components_split.append(re.split("\s+", components[i]))
    fractions_split.append([float(x) for x in re.split("\s+", fractions[i])])

    comp = [eval(s) for s in components_split[i]]
    ele_frac = fractions_split[i]

    calculator = FeatureCalculator(comp, ele_frac)
    features_temp = calculator.calculate_features()
    features[i, :] = features_temp

# Write to dataframe
T_features = pd.DataFrame(
    features,
    columns=[
        "a",
        "delta_a",
        "Tm",
        "sigma_Tm",
        "Hmix",
        "sigma_Hmix",
        "ideal_S",
        "elec_nega",
        "sigma_elec_nega",
        "VEC",
        "sigma_VEC",
        "bulk_modulus",
        "sigma_bulk_modulus",
    ],
)

T_features.to_excel(T_fname, index=False, columns=Features_Sel)

# Pearson's correlation
corr = T_features.corr()
plt.figure(figsize=(12, 12))
sns.heatmap(corr, annot=True)
plt.title("Correlation Matrix for hardness Features")
plt.show()